In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#reading in the train_df
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
train_df[train_df.Id == 'de4c8260-cd87-431c-b425-70c9c44596d0']

In [ ]:
import os
#filenames of all the training json files 
fnames = os.listdir('../input/coleridgeinitiative-show-us-the-data/train')
len(fnames)

In [ ]:
from tqdm import tqdm
mydataframe = pd.DataFrame()

paragraph_num = []

for file in tqdm(fnames):
    #print('-'*52)
    paper_path = os.path.join('../input/coleridgeinitiative-show-us-the-data/train', file)
    
    paper = pd.read_json(paper_path)
    paper_id = file.replace('.json','')
    
    find = train_df[train_df.Id == paper_id].dataset_label.unique()
   
    section_count = 0 
    small = []
    #print(paper)
    for text in paper.iterrows(): #each section
        section_count += 1
        text_df = text[1]
        for find_label in find:
            if find_label.lower() in text_df.text.lower():
                for sent in text_df.text.split('.'): #for each sentence in section
                    if find_label.lower() in sent.lower():
                        mydataframe = mydataframe.append({'Id':paper_id,'label_found':find_label.lower(),'section_title':text_df.section_title,'section_num':section_count,'total_section':len(paper),'sentence':sent,}, ignore_index=True)
                small.append(text_df.text[0])
    if small == []:#if i havnt found any find labels in the whole paper
        mydataframe = mydataframe.append({'Id':paper_id,'label_found':find_label.lower(),'section_title':'NONE','section_num':'NONE','total_section':len(paper),'sentence':'NONE',}, ignore_index=True)
 
    paragraph_num.append(small)
    
mydataframe

    

In [ ]:
#how many papers did i not find the label in the text
count = 0 
for i,v in enumerate(paragraph_num):
    if v == []:
        count += 1
        
count

In [ ]:
#removes the label from the sentence 
c_sent = []
for row in mydataframe.iterrows():
    temp = row[1].sentence.lower()
    for word in row[1].label_found.split(' '):
        temp = temp.replace(word,'')
    c_sent.append(temp)
    
    
mydataframe['clean_sentence'] = c_sent

In [ ]:
# reading csv files and train & test file paths
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_files_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_files_path = '../input/coleridgeinitiative-show-us-the-data/test'

In [ ]:
import string
import re
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = text.replace('-',' ')
    text = ''.join([k for k in text if k not in string.punctuation])
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    text = re.sub(' +', ' ', text)
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

In [ ]:
def read_append_return(filename, train_files_path=train_files_path, output='text'):
    """
    Function to read json file and then return the text data from them and append to the dataframe
    """
    json_path = os.path.join(train_files_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [ ]:
%%time
import json
tqdm.pandas()   #tqdm is used to show any code running with a progress bar. 
train_df['text'] = train_df['Id'].progress_apply(read_append_return)

In [ ]:
train_df.cleaned_label.unique()

In [ ]:
%%time
tqdm.pandas()
train_df['text'] = train_df['text'].progress_apply(text_cleaning)

In [ ]:
mydataframe[mydataframe.section_num == 'NONE'].Id.unique()
train_df[train_df.Id == 'de4c8260-cd87-431c-b425-70c9c44596d0']
empty = mydataframe[mydataframe.section_num == 'NONE'].Id.unique()
l = train_df.Id.apply(lambda x: x in empty)
didnt_find_df = train_df[l]#.groupby('cleaned_label').count()
didnt_find_df

In [ ]:
c = 0
for i in range(len(didnt_find_df)):
    if didnt_find_df.iloc[i].cleaned_label in didnt_find_df.iloc[i].text:
        train_df[train_df.Id == didnt_find_df.iloc[i].Id]
        paper_path = os.path.join('../input/coleridgeinitiative-show-us-the-data/train', str(didnt_find_df.iloc[i].Id)+'.json')
        paper = pd.read_json(paper_path)
        #print(len(paper))
        sent
        c += 1
    else:
        pass
        #print(didnt_find_df.iloc[i].Id)
        #print(train_df[train_df.Id == didnt_find_df.iloc[i].Id])
        #print(list(train_df[train_df.Id == didnt_find_df.iloc[i].Id].text))
        #break
 #       print({'Id':didnt_find_df.iloc[i].Id,'label_found':didnt_find_df.iloc[i].cleaned_label,'section_title':text_df.section_title,'section_num':section_count,'total_section':len(paper),'sentence':sent})#, ignore_index=True)
        #mydataframe
c
#NOAA water-level station

In [ ]:
from tqdm import tqdm
mydataframe = pd.DataFrame()

paper_datasets = []
fnames = os.listdir('../input/coleridgeinitiative-show-us-the-data/test')
for file in tqdm(fnames):
    #print('-'*52)
    paper_path = os.path.join('../input/coleridgeinitiative-show-us-the-data/test', file)
    
    paper = pd.read_json(paper_path)
    paper_id = file.replace('.json','')
    
    find = train_df.dataset_label.unique()
   
    section_count = 0 
    small = []
    #print(paper)
    for text in paper.iterrows(): #each section
        section_count += 1
        text_df = text[1]
        for find_label in find:
            if find_label.lower() in text_df.text.lower():
                small.append(find_label.lower())
    if small == []:#if i havnt found any find labels in the whole paper
        small.append('data set')
    else:
        small = " | ".join(set(small))
 
    paper_datasets.append({'Id':paper_id, 'PredictionString':small})
sub_df = pd.DataFrame(paper_datasets)

In [ ]:
sub_df.to_csv('./submission.csv', index=False)


In [ ]:
pd.read_csv('./submission.csv')